# Visualization
This notebooks serves to visualize results of an individual run in detail.

In [ ]:
# core stuff
import gravann,os
import numpy as np
import pickle as pk
from tqdm import tqdm
import torch

# For animation etc.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython import display
import matplotlib.pyplot as plt
import imageio

# Ensure that changes in imported module (gravann most importantly) are autoreloaded
%load_ext autoreload
%autoreload 2

# If possible enable CUDA
gravann.enableCUDA()
gravann.fixRandomSeeds()
device = os.environ["TORCH_DEVICE"]
print("Will use device ",device)

# Define the run folder

In [ ]:
results_folder = "results\differential_training_hollow2_ACC_siren_diff_train\Hollow2.pk\LR=0.0001_loss=normalized_L1_loss_encoding=direct_encoding_batch_size=1000_target_sample=spherical_activation=Tanh_omega=3e+01/"
differential_training = True

model, encoding, sample, c, use_acc, mascon_points, mascon_masses_u, mascon_masses_nu, cfg = gravann.load_model_run(
        results_folder, differential_training)

torch.cuda.empty_cache()


In [ ]:
c

In [ ]:
gravann.plot_model_vs_mascon_contours(model,encoding,mascon_points,mascon_masses_u,c=c,heatmap=True,save_path="figures/"+sample+".png",mascon_alpha=0.175);

In [ ]:
#differential
gravann.plot_model_vs_mascon_contours(model,encoding,mascon_points,mascon_masses_u,c=c,heatmap=True,save_path="figures/"+sample+".png",mascon_alpha=0.175,add_shape_base_value="3dmeshes/"+sample);

In [ ]:
gravann.plot_model_mascon_acceleration("3dmeshes/"+sample,model,encoding,mascon_points,mascon_masses_u,plane="XY", altitude=0.1,N=7500,c=c,logscale=False);

In [ ]:
gravann.plot_model_mascon_acceleration("3dmeshes/"+sample,model,encoding,mascon_points,mascon_masses_u,plane="XY", altitude=0.1,N=7500,c=c,logscale=False,save_path="figures/acc_xy.png");
gravann.plot_model_mascon_acceleration("3dmeshes/"+sample,model,encoding,mascon_points,mascon_masses_u,plane="XZ", altitude=0.1,N=7500,c=c,logscale=False,save_path="figures/acc_xz.png");
gravann.plot_model_mascon_acceleration("3dmeshes/"+sample,model,encoding,mascon_points,mascon_masses_u,plane="YZ", altitude=0.1,N=7500,c=c,logscale=False,save_path="figures/acc_yz.png");

In [ ]:
mascon_alpha = 0
images = []
for offset in tqdm(np.linspace(-0.75,0.75,10)):
    fig = gravann.plot_model_vs_mascon_contours(model,encoding,mascon_points,mascon_masses,c=c,offset=offset,heatmap=True,mascon_alpha=mascon_alpha);
    fig.canvas.draw();
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    images.append(image)
    plt.close("all")

In [ ]:
gifPath = "gifs/contourf.gif"
imageio.mimsave(gifPath, images)

In [ ]:
# Display GIF in Jupyter, CoLab, IPython
with open(gifPath,'rb') as f:
    display.Image(data=f.read(), format='png')

## Altitude Sampler

In [ ]:
sampler = gravann.get_target_point_sampler(1000,"altitude",bounds=[0.1,0.1],limit_shape_to_asteroid="3dmeshes/Eros_lp.pk")